# Test of Structural Mechanics
This nodebook is based on the test `test_structural_mechanics_model_bernoulli_beam_2.cc`.
It shows how meshes can be read in from the file system.

In [1]:
try:
    import pyakantu as pyaka
except:
    print("Error importing pyakantu, try the other one")
    import py11_akantu as pyaka
    
import copy
import numpy

Error importing pyakantu, try the other one


In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

### Creating the Mesh
We will read it in from a file directly, instead of manually create it.

In [3]:
# Create a mesh for the two dimensional case
beam = pyaka.Mesh(2)
beam.read("_bernoulli_beam_2.msh", pyaka._miot_gmsh_struct)

### Creating the Model

In [4]:
model = pyaka.StructuralMechanicsModel(beam)

#### Setting up the Modell

##### Creating and Inserting the Materials

In [5]:
mat1 = pyaka.StructuralMaterial()
mat1.E = 3e10
mat1.I = 0.0025
mat1.A = 0.01
mat1ID = model.addMaterial(mat1)

In [6]:
mat2 = pyaka.StructuralMaterial()
mat2.E = 3e10
mat2.I = 0.00128
mat2.A = 0.01
mat2ID = model.addMaterial(mat2)

##### Initializing the Model

In [7]:
model.initFull(pyaka.AnalysisMethod._static)

##### Assigning the Materials

In [8]:
materials = model.getElementMaterialMap(pyaka.ElementType._bernoulli_beam_2)

In [9]:
materials[0][0] = mat1ID
materials[1][0] = mat2ID

##### Setting Boundaries

In [10]:
# Neumann
M = 3600
q = 6000
L = 10
forces = model.getExternalForce()
forces[:] = 0
forces[0, 1] = -q * L / 2
forces[0, 2] = -q * L * L / 12
forces[1, 1] = -q * L / 2
forces[1, 2] = q * L * L / 12
forces[2, 0] = mat2.E * mat2.A / 18 
forces[2, 2] = -M

In [11]:
# Dirichlets
# Block all dofs of the first node, since it is fixed.
#  All other nodes have no restrictions
boundary = model.getBlockedDOFs()
boundary[0, :] = True
boundary[1, :] = False
boundary[1, 1] = True
boundary[2, :] = False
boundary[2, 1] = True

### Solving the System

In [12]:
model.solveStep()
disp = model.getDisplacement()

Extract the solution

In [13]:
d1 = disp[1, 2]
d2 = disp[2, 2]
d3 = disp[1, 0]


### Compare with Analytical Solution
For reference see the referenced file above.

In [14]:
d1_ex = 5.6 / 4800
d2_ex = -3.7 / 4800
d3_ex = 10. / 18

In [15]:
from math import isclose 

In [16]:
status = True
if not isclose(d1_ex, d1):
    print("Failed for d1, expected '{}' but got '{}'".format(d1_ex, d1))
    status = False
if not isclose(d2_ex, d2):
    print("Failed for d2, expected '{}' but got '{}'".format(d2_ex, d2))
    status = False
if not isclose(d3_ex, d3):
    print("Failed for d3, expected '{}' but got '{}'".format(d3_ex, d3))
    status = False
if status:
    print("It worked")
else:
    raise ValueError("There is an error, some tests failed.")

It worked
